In [ ]:
# Reikalingos bibliotekos

!pip install biopython
from Bio.Seq import Seq
from Bio import SeqIO
from google.colab import drive
drive.mount('/content/drive/')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 7.7 MB/s 
Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Funkcijos visų galimų kodonų ir dikodonų sąrašams sugeneruoti.

def generateAllPossibleCodons():
  codons = []
  letters = ['A','T','C','G']
  for first_letter in letters:
    for second_letter in letters:
      for third_letter in letters:
        codons.append(first_letter + second_letter + third_letter)
  return codons

def generateAllPossibleDicodons():
    all_codons = generateAllPossibleCodons()
    dicodons = []
    
    for first_codon in all_codons:
      for second_codon in all_codons:
        dicodons.append(first_codon + second_codon)

    return dicodons

In [ ]:
# Konstantinės reikšmės

START_CODONS = ['ATG']
END_CODONS = ['TAG', 'TAA', 'TGA']
ALL_POSSIBLE_CODONS = generateAllPossibleCodons()
ALL_POSSIBLE_DICODONS = generateAllPossibleDicodons()

In [ ]:
# Funkcija, skirta surasti start ir stop kodonų pozicijas sekoje. Ieškant start ir stop kodonų pozicijų, atsižvelgiama į skaitymo rėmelį.

def findCodonPositions(sequence):
  sequence_length = len(sequence)

  start_codons = {0:[], 1:[], 2:[]}
  end_codons = {0:[], 1:[], 2:[]}

  i = 0
  frame = 0

  while i < sequence_length - 2:
    possible_codon = str(sequence[i:i + 3])

    if possible_codon in START_CODONS:
      start_codons.get(frame).append(i)

    if possible_codon in END_CODONS:
      end_codons.get(frame).append(i + 3)

    i+=1
    frame = (frame + 1) % 3

  return start_codons, end_codons

In [ ]:
# Funkcija, skirta rasti sekos fragmentams, kurios prasideda start ir baigiasi stop kodonu.

def findFragments(start_codon_positions, end_codon_positions):

  codon_pairs = []

  for start in start_codon_positions:
    nearest_end_codon = max(end_codon_positions)
    for end in end_codon_positions:
      if start < end and end < nearest_end_codon:
        nearest_end_codon = end
    if(start < nearest_end_codon):
      codon_pairs.append((start,nearest_end_codon))
  return codon_pairs

In [ ]:
# Funkcija, skirta surasti ilgiausius sekos fragmentus. Trumpesni fragmentai, kurie patenka į ilgesnius fragmentus yra atmetami.

def findLongestFragments(codon_pair_list):
  unique_stop_codon_positions = []
  for codon_pair in codon_pair_list:
    unique_stop_codon_positions.append(codon_pair[1])
  unique_stop_codon_positions = list(set(unique_stop_codon_positions))

  longest_codon_pairs = []
  for i in range(len(codon_pair_list)):
    longest_codon_pair = codon_pair_list[i]
    if longest_codon_pair[1] in unique_stop_codon_positions:
      unique_stop_codon_positions.remove(longest_codon_pair[1])
      for j in range(i+1,(len(codon_pair_list))):
        if longest_codon_pair[1] == codon_pair_list[j][1]:
          if longest_codon_pair[0] > codon_pair_list[j][0]:
            longest_codon_pair = codon_pair_list[j]
      longest_codon_pairs.append(longest_codon_pair)
    else: 
      continue
  return longest_codon_pairs

In [ ]:
# Funkcija, skirta konvertuoti fragmentų pradžios ir pabaigos indeksų poras į fragmentus, sudarytus iš raidžių.

def numericFragmentToAlphabeticFragment(sequence, sequence_fragments):
  sequence_strings = []
  for sequence_fragment in sequence_fragments:
    sequence_strings.append(str(sequence[sequence_fragment[0]:sequence_fragment[1]])[3:-3])
  return sequence_strings

In [ ]:
# Funkcija, skirta atmesti trumpus fragmentus.


def filterShortFragments(sequence_fragments):
  filtered_fragments = []
  for sequence_fragment in sequence_fragments:
    if(len(sequence_fragment) > 106):
      filtered_fragments.append(sequence_fragment)
  return filtered_fragments

In [ ]:
# Funkcijos, skirtos apskaičiuoti kodonų ir dikodonų dažniams vienoje sekoje.

def findCodonsFrequency(sequence_fragments):
  frequencies = {}

  for codon in ALL_POSSIBLE_CODONS:
      frequencies[codon] = 0

  codonsCount = 0
  i = 0

  for sequence_fragment in sequence_fragments:
    while i < len(sequence_fragment) - 2:
      codon = sequence_fragment[i:i + 3]
      frequencies[codon] += 1
      codonsCount += 1
      i += 1

  for codon in ALL_POSSIBLE_CODONS:
      frequencies[codon] = round(frequencies[codon] / codonsCount, 3)

  return frequencies

def findDicodonsFrequency(sequence_fragments):
  frequencies = {}

  for dicodon in ALL_POSSIBLE_DICODONS:
      frequencies[dicodon] = 0

  dicodonsCount = 0
  i = 0

  for sequence_fragment in sequence_fragments:
    while i < len(sequence_fragment) - 5:
      codon = sequence_fragment[i:i + 6]
      frequencies[codon] += 1
      i += 1
      dicodonsCount += 1

  for dicodon in ALL_POSSIBLE_DICODONS:
      frequencies[dicodon] = round(frequencies[dicodon] / dicodonsCount, 3)

  return frequencies

In [ ]:
# Funkcijos, skirtos palyginti kodonų ir dikodonų dažnius tarp skirtingų sekų.

def compareCodonFrequencies(frequency1, frequency2):
  distance = 0
  for i, codon in enumerate(ALL_POSSIBLE_CODONS):
    distance += abs(frequency1[codon] - frequency2[codon])
  return distance


def compareDicodonFrequencies(frequency1, frequency2):
  distance = 0
  for i, dicodon in enumerate(ALL_POSSIBLE_DICODONS):
    distance += abs(frequency1[dicodon] - frequency2[dicodon])
  return distance

In [ ]:
def calculateFrequencies(sequence):
  sequence_reverse_complement = sequence.reverse_complement()

  #Susirandame abiejų sekų start ir stop kodonus

  sequence_start_codons, sequence_end_codons = findCodonPositions(str(sequence.seq))
  sequence_reverse_complement_start_codons, sequence_reverse_complement_end_codons = findCodonPositions(str(sequence_reverse_complement.seq))

  # Susirandame abiejų sekų visus fragmentus, prasidedančius START ir pasibaigiančius STOP kodonu.

  sequence_fragments_frame0 = findFragments(sequence_start_codons.get(0), sequence_end_codons.get(0))
  sequence_reverse_complement_fragments_frame0 = findFragments(sequence_reverse_complement_start_codons.get(0), sequence_reverse_complement_end_codons.get(0))

  sequence_fragments_frame1 = findFragments(sequence_start_codons.get(1), sequence_end_codons.get(1))
  sequence_reverse_complement_fragments_frame1 = findFragments(sequence_reverse_complement_start_codons.get(1), sequence_reverse_complement_end_codons.get(1))

  sequence_fragments_frame2 = findFragments(sequence_start_codons.get(2), sequence_end_codons.get(2))
  sequence_reverse_complement_fragments_frame2 = findFragments(sequence_reverse_complement_start_codons.get(2), sequence_reverse_complement_end_codons.get(2))

  # Surandame fragmentus, kuriu stop kodonas yra toliausiai nutoles nuo start kodono.

  sequence_fragments_frame0 = findLongestFragments(sequence_fragments_frame0)
  sequence_reverse_complement_fragments_frame0 = findLongestFragments(sequence_reverse_complement_fragments_frame0)

  sequence_fragments_frame1 = findLongestFragments(sequence_fragments_frame1)
  sequence_reverse_complement_fragments_frame1 = findLongestFragments(sequence_reverse_complement_fragments_frame1)

  sequence_fragments_frame2 = findLongestFragments(sequence_fragments_frame2)
  sequence_reverse_complement_fragments_frame2 = findLongestFragments(sequence_reverse_complement_fragments_frame2)

  # Fragmentus paverčiame į jų tekstines išraiškas.

  sequence_fragments_frame0 = numericFragmentToAlphabeticFragment(str(sequence.seq), sequence_fragments_frame0)
  sequence_reverse_complement_fragments_frame0 = numericFragmentToAlphabeticFragment(str(sequence_reverse_complement.seq), sequence_reverse_complement_fragments_frame0)

  sequence_fragments_frame1 = numericFragmentToAlphabeticFragment(str(sequence.seq), sequence_fragments_frame1)
  sequence_reverse_complement_fragments_frame1 = numericFragmentToAlphabeticFragment(str(sequence_reverse_complement.seq), sequence_reverse_complement_fragments_frame1)

  sequence_fragments_frame2 = numericFragmentToAlphabeticFragment(str(sequence.seq), sequence_fragments_frame2)
  sequence_reverse_complement_fragments_frame2 = numericFragmentToAlphabeticFragment(str(sequence_reverse_complement.seq), sequence_reverse_complement_fragments_frame2)

  all_fragments = sequence_fragments_frame0 + sequence_fragments_frame1 + sequence_fragments_frame2 + sequence_reverse_complement_fragments_frame0 + sequence_reverse_complement_fragments_frame1 + sequence_reverse_complement_fragments_frame2

  all_fragments = filterShortFragments(all_fragments)

  return {'codon_frequencies': findCodonsFrequency(all_fragments), 'dicodon_frequencies': findDicodonsFrequency(all_fragments)}

In [ ]:
# Nuskaitome sekas ir apskaičiuojame jų kodonų ir dikodonų dažnius.

data = [{'filename':'bacterial1.fasta','virus':'Lactococcus_phage'},
        {'filename':'bacterial2.fasta','virus':'KM389305.1'},
        {'filename':'bacterial3.fasta','virus':'NC_028697.1'},
        {'filename':'bacterial4.fasta','virus':'KC821626.1'},
        {'filename':'mamalian1.fasta','virus':'coronavirus'},
        {'filename':'mamalian2.fasta','virus':'adenovirus'},
        {'filename':'mamalian3.fasta','virus':'U18337.1'},
        {'filename':'mamalian4.fasta','virus':'herpesvirus'}]

frequencies = {}

for data_element in data:
  sequence = SeqIO.read("/content/drive/MyDrive/bio_lab_1/" + data_element['filename'], "fasta")
  frequencies[data_element['virus']] = calculateFrequencies(sequence)

In [ ]:
# Atliekame palyginimą pagal kodonų dažnius.

for virus1 in data:
  print(virus1['virus'],end = " ")
  for virus2 in data:
    print(round(compareCodonFrequencies(frequencies[virus1['virus']]['codon_frequencies'],frequencies[virus2['virus']]['codon_frequencies']),3),end = " ")
  print()

Lactococcus_phage 0.0 0.468 0.389 0.389 0.368 0.54 0.425 0.828 
KM389305.1 0.468 0.0 0.421 0.605 0.462 0.298 0.599 0.536 
NC_028697.1 0.389 0.421 0.0 0.468 0.333 0.429 0.552 0.743 
KC821626.1 0.389 0.605 0.468 0.0 0.385 0.667 0.382 0.993 
coronavirus 0.368 0.462 0.333 0.385 0.0 0.518 0.415 0.802 
adenovirus 0.54 0.298 0.429 0.667 0.518 0.0 0.685 0.506 
U18337.1 0.425 0.599 0.552 0.382 0.415 0.685 0.0 0.927 
herpesvirus 0.828 0.536 0.743 0.993 0.802 0.506 0.927 0.0 


In [ ]:
# Atliekame palyginimą pagal dikodonų dažnius.

for virus1 in data:
  print(virus1['virus'],end = " ")
  for virus2 in data:
    print(round(compareDicodonFrequencies(frequencies[virus1['virus']]['dicodon_frequencies'],frequencies[virus2['virus']]['dicodon_frequencies']),3),end = " ")
  print()

Lactococcus_phage 0.0 1.689 1.17 1.044 0.982 1.267 1.282 1.877 
KM389305.1 1.689 0.0 1.801 1.827 1.723 1.726 1.875 2.022 
NC_028697.1 1.17 1.801 0.0 1.23 1.06 1.333 1.432 1.971 
KC821626.1 1.044 1.827 1.23 0.0 0.872 1.323 1.258 2.009 
coronavirus 0.982 1.723 1.06 0.872 0.0 1.167 1.192 1.853 
adenovirus 1.267 1.726 1.333 1.323 1.167 0.0 1.493 1.73 
U18337.1 1.282 1.875 1.432 1.258 1.192 1.493 0.0 2.069 
herpesvirus 1.877 2.022 1.971 2.009 1.853 1.73 2.069 0.0 
